### Description

This script aims to select and subset EN4 profile data for comparing to HYCOM. The data is downloaded from: https://hadleyserver.metoffice.gov.uk/en4/download-en4-2-1.html and a description of the profile files can be found here: https://hadleyserver.metoffice.gov.uk/en4/en4-0-2-profile-file-format.html

The files need to be prepared for ingesting by Bjorn's scripts to create station and depthlevels files (https://gitlab.com/backeb/hycom_enoi/-/blob/master/scripts/hycom/sandbox/make_hyc2station_infiles.py).

The goal is to:
1. first try subset the profiles to the model domain; then
2. try export the required data into a .txt (or .csv) file for generating the station and depth files

*Alternatively,* an attempt could be made to generate the files directly from the profiles netcdf files without first writing to a .txt file.

In [1]:
import glob
import pandas as pd
import numpy as np
import xarray as xr

In [2]:
# loading a profile data file
EN4_profile = xr.open_dataset('../Data/EN4_profiles/EN.4.2.1.f.profiles.g10.200901.nc')
# ds_EN4 = ds_EN4.sel(time=slice('2009-01','2014-04'))
# ds_EN4['temperature'] = ds_EN4['temperature'] - 273.15

'''
We now need to import all the files, or a list thereof for sequential preprocessing
in a loop
'''

'\nWe now need to import all the files, or a list thereof for sequential preprocessing\nin a loop\n'

In [54]:
test = EN4_profile['POTM_CORRECTED_QC'].where(EN4_profile['POTM_CORRECTED_QC']==b'4')
# (EN4_profile['POTM_CORRECTED_QC']).count()

In [55]:
test.count()

<xarray.DataArray 'POTM_CORRECTED_QC' ()>
array(206471)

In [41]:
# test.count(skipna=True)
# test['POTM_CORRECTED_QC'].count()
new_test = test.dropna('N_PROF')
new_test

<xarray.DataArray 'POTM_CORRECTED_QC' (N_PROF: 0, N_LEVELS: 400)>
array([], shape=(0, 400), dtype=object)
Dimensions without coordinates: N_PROF, N_LEVELS
Attributes:
    long_name:    quality on pot. temperature
    conventions:  q where q =[0,9]
    _fillvalue:   0

In [33]:
EN4_profile

<xarray.Dataset>
Dimensions:                       (N_CALIB: 1, N_HISTORY: 0, N_LEVELS: 400, N_PARAM: 5, N_PROF: 32595)
Dimensions without coordinates: N_CALIB, N_HISTORY, N_LEVELS, N_PARAM, N_PROF
Data variables:
    CALIBRATION_DATE              (N_PROF, N_CALIB, N_PARAM) |S14 ...
    CYCLE_NUMBER                  (N_PROF) int32 -2147483647 ... -2147483647
    DATA_CENTRE                   (N_PROF) |S2 b'MO' b'MO' b'MO' ... b'MO' b'MO'
    DATA_MODE                     (N_PROF) |S1 b'D' b'D' b'D' ... b'D' b'D' b'D'
    DATA_STATE_INDICATOR          (N_PROF) |S4 b'2C+ ' b'2C+ ' ... b'2C+ '
    DATA_TYPE                     |S16 b'ENSEMBLES EN3 v1'
    DATE_CREATION                 |S14 b'20170421133031'
    DATE_UPDATE                   |S14 b'20170421133031'
    DC_REFERENCE                  (N_PROF) |S16 b' A20090101-02729' ... b' A20090131-65980'
    DEPH_CORRECTED                (N_PROF, N_LEVELS) float32 ...
    DEPH_CORRECTED_QC             (N_PROF, N_LEVELS) |S1 b'' b'' b'' ... b'' b''
    DIRECTION                     (N_PROF) |S1 b'' b'' b'' b'' ... b'' b'' b''
    FORMAT_VERSION                |S4 b'2.0'
    HANDBOOK_VERSION              float32 1.0
    HISTORY_ACTION                (N_HISTORY, N_PROF) |S4 
    HISTORY_DATE                  (N_HISTORY, N_PROF) |S14 
    HISTORY_INSTITUTION           (N_HISTORY, N_PROF) |S4 
    HISTORY_PARAMETER             (N_HISTORY, N_PROF) |S4 
    HISTORY_PREVIOUS_VALUE        (N_HISTORY, N_PROF) float32 
    HISTORY_QCTEST                (N_HISTORY, N_PROF) |S16 
    HISTORY_SOFTWARE              (N_HISTORY, N_PROF) |S4 
    HISTORY_SOFTWARE_RELEASE      (N_HISTORY, N_PROF) |S4 
    HISTORY_START_DEPH            (N_HISTORY, N_PROF) float32 
    HISTORY_STOP_DEPH             (N_HISTORY, N_PROF) float32 
    INST_REFERENCE                (N_PROF) |S64 b'                                                             846' ... b'                                                           22140'
    JULD                          (N_PROF) datetime64[ns] 2009-01-01T23:43:08 ... 2009-01-31T23:16:48
    JULD_LOCATION                 (N_PROF) datetime64[ns] 2009-01-01T23:43:08 ... 2009-01-31T23:16:48
    JULD_QC                       (N_PROF) |S1 b'' b'' b'' b'' ... b'' b'' b''
    LATITUDE                      (N_PROF) float64 -65.42 56.34 ... 21.9 -23.77
    LONGITUDE                     (N_PROF) float64 -39.74 -17.4 ... -156.7 110.8
    PARAMETER                     (N_PROF, N_CALIB, N_PARAM) |S4 ...
    PI_NAME                       (N_PROF) |S64 b'Simon Good, Met Office                                          ' ... b'Simon Good, Met Office                                          '
    PLATFORM_NUMBER               (N_PROF) |S8 b'5901738 ' ... b'6299    '
    POSITIONING_SYSTEM            (N_PROF) |S8 b'' b'' b'' b'' ... b'' b'' b''
    POSITION_QC                   (N_PROF) |S1 b'1' b'1' b'1' ... b'1' b'1' b'1'
    POTM_CORRECTED                (N_PROF, N_LEVELS) float32 ...
    POTM_CORRECTED_QC             (N_PROF, N_LEVELS) |S1 b'1' b'1' ... b'0' b'0'
    PROFILE_DEPH_QC               (N_PROF) |S1 b'' b'' b'' b'' ... b'' b'' b''
    PROFILE_POTM_QC               (N_PROF) |S1 b'1' b'1' b'1' ... b'1' b'1' b'1'
    PROFILE_PSAL_QC               (N_PROF) |S1 b'1' b'1' b'1' ... b'4' b'4' b'4'
    PROJECT_NAME                  (N_PROF) |S64 b'ARGOD                                                           ' ... b'WOD09XBTCY    9699                                              '
    PSAL_CORRECTED                (N_PROF, N_LEVELS) float32 ...
    PSAL_CORRECTED_QC             (N_PROF, N_LEVELS) |S1 ...
    QC_FLAGS_LEVELS               (N_PROF, N_LEVELS) int32 ...
    QC_FLAGS_PROFILES             (N_PROF) int32 0 0 0 0 0 0 ... 0 0 0 0 0
    REFERENCE_DATE_TIME           |S14 b'19500101000000'
    SCIENTIFIC_CALIB_COEFFICIENT  (N_PROF, N_CALIB, N_PARAM) |S256 ...
    SCIENTIFIC_CALIB_COMMENT      (N_PROF, N_CALIB, N_PARAM) |S256 ...
    SCIENTIFIC_CALIB_EQUATION     (N_PROF, N_CALIB, N_P

In [14]:
'''
This converting to dataframe will need to take place within the loop that writes all
data into a single file or variable.
'''

# convert the required fields to dataframe
n_prof = EN4_profile['N_PROF'].to_dataframe()
juld = EN4_profile['JULD'].to_dataframe()
lat = EN4_profile['LATITUDE'].to_dataframe()
lon = EN4_profile['LONGITUDE'].to_dataframe()
sal = EN4_profile['PSAL_CORRECTED'].to_dataframe()
temp = EN4_profile['TEMP'].to_dataframe()
depth = EN4_profile['DEPH_CORRECTED'].to_dataframe()

# subset the lats and lons to model domain
lat_ind = np.where((lat <= -10) & (lat >= -50 ))[0]
lat = lat.iloc[lat_ind]
lon_ind = np.where((lon <= 70) & (lon >= 0))[0]
lon = lon.iloc[lon_ind]

In [15]:
# Join lats and lons for first dataset, 'tester'
tester = lat.join(lon, how='inner')

In [16]:
# join other dataframes with inner join (Use intersection of keys from both frames)
tester2 = tester.join(depth,how='inner').join(temp, how='inner').join(sal, how='inner').join(juld, how='inner')
tester2

LATITUDE  LONGITUDE  DEPH_CORRECTED   TEMP  PSAL_CORRECTED  \
N_PROF N_LEVELS                                                                
21     0        -43.943001     10.397        4.959837  7.933       34.209751   
       1        -43.943001     10.397        9.919554  7.851       34.218380   
       2        -43.943001     10.397       14.879150  7.834       34.221432   
       3        -43.943001     10.397       19.838627  7.814       34.225231   
       4        -43.943001     10.397       24.797983  7.698       34.236179   
...                    ...        ...             ...    ...             ...   
32588  395      -29.983000     13.367             NaN    NaN             NaN   
       396      -29.983000     13.367             NaN    NaN             NaN   
       397      -29.983000     13.367             NaN    NaN             NaN   
       398      -29.983000     13.367             NaN    NaN             NaN   
       399      -29.983000     13.367             NaN    NaN             NaN   

                                      JULD  
N_PROF N_LEVELS                             
21     0        2009-01-01 13:34:46.000017  
       1        2009-01-01 13:34:46.000017  
       2        2009-01-01 13:34:46.000017  
       3        2009-01-01 13:34:46.000017  
       4        2009-01-01 13:34:46.000017  
...                                    ...  
32588  395      2009-01-31 22:39:00.000008  
       396      2009-01-31 22:39:00.000008  
       397      2009-01-31 22:39:00.000008  
       398      2009-01-31 22:39:00.000008  
       399      2009-01-31 22:39:00.000008  

[294000 rows x 6 columns]

In [7]:
# Create list of index values to be referenced later
tester2.index[0::400]
lst = [i[0] for i in tester2.index[0::400]]
lst[0]

'''
Hereafter, removal of NaN depths should be considered, then the building of the
subsequent months of profile data should be done, too.
The subsequent months should have unqiue station values, too, so the current thinking
is to add len(N_PROF) to the next months N_PROF consecutively.
'''

NameError: name 'tester2' is not defined

In [107]:
# Testing retrieval of max depth value
tester2.loc[lst[0]]['DEPH_CORRECTED'].max()

'''
Here the max value can be retrieved, but the advantage of having each depth value
in the final dataframe is that the entire depthvalues file can match the depth
values from the profile, as opposed to Bjorn's 5 m incremental approach.
'''

1876.2327

In [38]:
EN4_profile

<xarray.Dataset>
Dimensions:                       (N_CALIB: 1, N_HISTORY: 0, N_LEVELS: 400, N_PARAM: 5, N_PROF: 32595)
Dimensions without coordinates: N_CALIB, N_HISTORY, N_LEVELS, N_PARAM, N_PROF
Data variables:
    CALIBRATION_DATE              (N_PROF, N_CALIB, N_PARAM) |S14 b'' ... b''
    CYCLE_NUMBER                  (N_PROF) int32 -2147483647 ... -2147483647
    DATA_CENTRE                   (N_PROF) |S2 b'MO' b'MO' b'MO' ... b'MO' b'MO'
    DATA_MODE                     (N_PROF) |S1 b'D' b'D' b'D' ... b'D' b'D' b'D'
    DATA_STATE_INDICATOR          (N_PROF) |S4 b'2C+ ' b'2C+ ' ... b'2C+ '
    DATA_TYPE                     |S16 b'ENSEMBLES EN3 v1'
    DATE_CREATION                 |S14 b'20170421133031'
    DATE_UPDATE                   |S14 b'20170421133031'
    DC_REFERENCE                  (N_PROF) |S16 b' A20090101-02729' ... b' A20090131-65980'
    DEPH_CORRECTED                (N_PROF, N_LEVELS) float32 4.3566914 ... nan
    DEPH_CORRECTED_QC             (N_PROF, N_LEVEL

## Building the loop

In [60]:
# Get list of profile netcdf files and initialise some variables and lists
profile_files = glob.glob('../Data/EN4_profiles/EN.4.2.1.f.profiles.g10.2009[0]1.nc')
metadata = pd.DataFrame()
total_profiles = 0
profile_list = []

# Subset profiles to model domain and collate required information in dataframe
for filename in profile_files:
    print(filename)
    ds = xr.open_dataset(filename)
    
    # increment N_PROF by previous total to ensure unique IDs
    ds['N_PROF'] = ds['N_PROF'] + total_profiles
    total_profiles = ds['N_PROF'][-1].values
    
    # convert the required fields to dataframes
    n_prof = ds['N_PROF'].to_dataframe()
    juld = ds['JULD'].to_dataframe()
    lat = ds['LATITUDE'].to_dataframe()
    lon = ds['LONGITUDE'].to_dataframe()
    sal = ds['PSAL_CORRECTED'].to_dataframe()
    salqc = ds['PSAL_CORRECTED_QC'].to_dataframe()
    temp = ds['TEMP'].to_dataframe()
    tempqc = ds['POTM_CORRECTED_QC'].to_dataframe()
    depth = ds['DEPH_CORRECTED'].to_dataframe()

    # subset the lats and lons to model domain
    lat_ind = np.where((lat <= -10) & (lat >= -50 ))[0]
    lat = lat.iloc[lat_ind]
    lon_ind = np.where((lon <= 70) & (lon >= 0))[0]
    lon = lon.iloc[lon_ind]
    
    # join dataframes with inner join (Use intersection of keys from both frames)
    file_metadata = lat.join(lon, how='inner').join(depth,how='inner').join(temp, how='inner').join(sal, how='inner').join(salqc, how='inner').join(tempqc, how='inner').join(juld, how='inner')
    profile_list.extend([i[0] for i in file_metadata.index[0::400]])
    file_metadata = file_metadata.dropna(subset=['DEPH_CORRECTED'])
    metadata = metadata.append(file_metadata)
    print(total_profiles)
    
    # optionally save the metadata file at this point to .csv or .txt

# Write the depthlevels.in and stations.in files using the metadata dataframe
# for profile in profile_list:
#     print(profile)
    
#     # Assign variables from profile data
#     stn = profile
#     date = pd.to_datetime(metadata.loc[profile]['JULD'][0])
#     doy = date.dayofyear - 1
#     year = date.year
#     lon = metadata.loc[profile]['LONGITUDE'][0]
#     lat = metadata.loc[profile]['LATITUDE'][0]
#     depthlevels = metadata.loc[profile]['DEPH_CORRECTED'].values
    
#     # Write depthlevels.in file
#     depth_file = open("EN4_depth_station/depthlevels.in.{0}.{1}_{2:03}".format(stn,year,doy), "w")
#     depth_file.write("{0}                  # Number of z levels\n".format(len(depthlevels)))
#     for depth in depthlevels:
#         depth_file.write("{0}\n".format(depth))
#     depth_file.close()
    
#     # Write stations.in file
#     station_file = open("EN4_depth_station/stations.in.{0}.{1}_{2:03}".format(stn,year,doy), "w")
#     station_file.write("#{0}\n".format(stn))
#     station_file.write("{0:.2f}   {1:.2f}\n".format(lon,lat))
#     station_file.close()

../Data/EN4_profiles/EN.4.2.1.f.profiles.g10.200901.nc
32594


In [61]:
file_metadata

LATITUDE  LONGITUDE  DEPH_CORRECTED       TEMP  \
N_PROF N_LEVELS                                                    
21     0        -43.943001     10.397        4.959837   7.933000   
       1        -43.943001     10.397        9.919554   7.851000   
       2        -43.943001     10.397       14.879150   7.834000   
       3        -43.943001     10.397       19.838627   7.814000   
       4        -43.943001     10.397       24.797983   7.698000   
...                    ...        ...             ...        ...   
32588  382      -29.983000     13.367      797.461426   4.331900   
       383      -29.983000     13.367      808.062195   4.261900   
       384      -29.983000     13.367      818.662842   4.181900   
       385      -29.983000     13.367      829.263550   4.121900   
       386      -29.983000     13.367      839.278992  27.211901   

                 PSAL_CORRECTED PSAL_CORRECTED_QC POTM_CORRECTED_QC  \
N_PROF N_LEVELS                                                       
21     0              34.209751              b'1'              b'1'   
       1              34.218380              b'1'              b'1'   
       2              34.221432              b'1'              b'1'   
       3              34.225231              b'1'              b'1'   
       4              34.236179              b'1'              b'1'   
...                         ...               ...               ...   
32588  382                  NaN              b'4'              b'1'   
       383                  NaN              b'4'              b'1'   
       384                  NaN              b'4'              b'1'   
       385                  NaN              b'4'              b'1'   
       386                  NaN              b'4'              b'4'   

                               JULD  
N_PROF N_LEVELS                      
21     0        2009-01-01 13:34:46  
       1        2009-01-01 13:34:46  
       2        2009-01-01 13:34:46  
       3        2009-01-01 13:34:46  
       4        2009-01-01 13:34:46  
...                             ...  
32588  382      2009-01-31 22:39:00  
       383      2009-01-31 22:39:00  
       384      2009-01-31 22:39:00  
       385      2009-01-31 22:39:00  
       386      2009-01-31 22:39:00  

[108636 rows x 8 columns]

In [59]:
file_metadata.loc[file_metadata['PSAL_CORRECTED_QC'] == b'4', 'PSAL_CORRECTED'] = np.nan
file_metadata.loc[file_metadata['POTM_CORRECTED_QC'] == b'4', 'TEMP'] = np.nan
file_metadata

LATITUDE  LONGITUDE  DEPH_CORRECTED    TEMP  PSAL_CORRECTED  \
N_PROF N_LEVELS                                                                 
21     0        -43.943001     10.397        4.959837  7.9330       34.209751   
       1        -43.943001     10.397        9.919554  7.8510       34.218380   
       2        -43.943001     10.397       14.879150  7.8340       34.221432   
       3        -43.943001     10.397       19.838627  7.8140       34.225231   
       4        -43.943001     10.397       24.797983  7.6980       34.236179   
...                    ...        ...             ...     ...             ...   
32588  382      -29.983000     13.367      797.461426  4.3319             NaN   
       383      -29.983000     13.367      808.062195  4.2619             NaN   
       384      -29.983000     13.367      818.662842  4.1819             NaN   
       385      -29.983000     13.367      829.263550  4.1219             NaN   
       386      -29.983000     13.367      839.278992     NaN             NaN   

                PSAL_CORRECTED_QC POTM_CORRECTED_QC                JULD  
N_PROF N_LEVELS                                                          
21     0                     b'1'              b'1' 2009-01-01 13:34:46  
       1                     b'1'              b'1' 2009-01-01 13:34:46  
       2                     b'1'              b'1' 2009-01-01 13:34:46  
       3                     b'1'              b'1' 2009-01-01 13:34:46  
       4                     b'1'              b'1' 2009-01-01 13:34:46  
...                           ...               ...                 ...  
32588  382                   b'4'              b'1' 2009-01-31 22:39:00  
       383                   b'4'              b'1' 2009-01-31 22:39:00  
       384                   b'4'              b'1' 2009-01-31 22:39:00  
       385                   b'4'              b'1' 2009-01-31 22:39:00  
       386                   b'4'              b'4' 2009-01-31 22:39:00  

[108636 rows x 8 columns]

In [10]:
metadata.loc[35200]

,LATITUDE,LONGITUDE,DEPH_CORRECTED,TEMP,PSAL_CORRECTED,JULD
N_LEVELS,,,,,,
0,-23.908,39.685,4.173264,29.375000,35.286011,2009-02-02 11:19:23
1,-23.908,39.685,8.942604,29.198000,35.281010,2009-02-02 11:19:23
2,-23.908,39.685,13.910549,28.910999,35.257011,2009-02-02 11:19:23
3,-23.908,39.685,18.977730,28.687000,35.248009,2009-02-02 11:19:23
4,-23.908,39.685,23.746723,28.570000,35.239010,2009-02-02 11:19:23
5,-23.908,39.685,29.012362,28.572001,35.244011,2009-02-02 11:19:23
6,-23.908,39.685,33.781124,28.495001,35.241009,2009-02-02 11:19:23
7,-23.908,39.685,39.046501,28.403000,35.215012,2009-02-02 11:19:23
8,-23.908,39.685,49.080154,27.454000,35.220009,2009-02-02 11:19:23


In [6]:
metadata.to_csv('SA_EN4_profiles_9.csv')

In [5]:
ds['N_PROF']

<xarray.DataArray 'N_PROF' (N_PROF: 32595)>
array([    0,     1,     2, ..., 32592, 32593, 32594])
Coordinates:
  * N_PROF   (N_PROF) int64 0 1 2 3 4 5 ... 32589 32590 32591 32592 32593 32594

## Writing to script

In [32]:
import glob
import pandas as pd
import numpy as np
import xarray as xr

profile_files_dir = '../Data/EN4_profiles/'
output_dir = 'EN4_depth_station/'

# Get list of profile netcdf files and initialise some variables and lists
profile_files = glob.glob('{0}/EN.4.2.1.f.profiles.g10.20090[1].nc'.format(profile_files_dir))
metadata = pd.DataFrame()
total_profiles = 0
profile_list = []

print('Collecting data from the following files: {0}'.format(profile_files))

# Subset profiles to model domain and collate required information in dataframe
for filename in profile_files:
    
    print('Now reading {0}...'.format(filename))
    
    ds = xr.open_dataset(filename)
    
    # increment N_PROF by previous total to ensure unique IDs
#     ds['N_PROF'] += total_profiles
    ds['N_PROF'] = ds['N_PROF'] + total_profiles
    total_profiles = ds['N_PROF'][-1].values
    
    # convert the required fields to dataframes
    n_prof = ds['N_PROF'].to_dataframe()
    juld = ds['JULD'].to_dataframe()
    lat = ds['LATITUDE'].to_dataframe()
    lon = ds['LONGITUDE'].to_dataframe()
    sal = ds['PSAL_CORRECTED'].to_dataframe()
    temp = ds['TEMP'].to_dataframe()
    depth = ds['DEPH_CORRECTED'].to_dataframe()

    # subset the lats and lons to model domain
    lat_ind = np.where((lat <= -10) & (lat >= -50 ))[0]
    lat = lat.iloc[lat_ind]
    lon_ind = np.where((lon <= 70) & (lon >= 0))[0]
    lon = lon.iloc[lon_ind]
    
    # join dataframes with inner join (Use intersection of keys from both frames)
    file_metadata = lat.join(lon, how='inner').join(depth, how='inner').join(temp, how='inner').join(sal, how='inner').join(juld, how='inner')
    profile_list.extend([i[0] for i in file_metadata.index[0::400]])
    file_metadata = file_metadata.dropna(subset=['DEPH_CORRECTED'])
    metadata = metadata.append(file_metadata)
    
    # optionally save the metadata file at this point to .csv or .txt

print('All data collected and stored in metadata variable.\nNow creating depthlevels.in and stations.in files...')
    
# Write the depthlevels.in and stations.in files using the metadata dataframe
for profile in profile_list:
    
    # Assign variables from profile data
    stn = profile
    date = pd.to_datetime(metadata.loc[profile]['JULD'][0])
    doy = date.dayofyear - 1
    year = date.year
    lon = metadata.loc[profile]['LONGITUDE'][0]
    lat = metadata.loc[profile]['LATITUDE'][0]
    depthlevels = metadata.loc[profile]['DEPH_CORRECTED'].values
    
    # Write depthlevels.in file
    depth_file = open("{0}/depthlevels.in.{1}.{2}_{3:03}".format(output_dir, stn, year, doy), "w")
    depth_file.write("{0}                  # Number of z levels\n".format(len(depthlevels)))
    for depth in depthlevels:
        depth_file.write("{0}\n".format(depth))
    depth_file.close()
    
    # Write stations.in file
    station_file = open("{0}/stations.in.{1}.{2}_{3:03}".format(output_dir, stn, year, doy), "w")
    station_file.write("#{0}\n".format(stn))
    station_file.write("{0:.2f}   {1:.2f}\n".format(lon, lat))
    station_file.close()

print('All profiles_files read. depthlevels.in and stations.in files produced successfully.')

../Data/EN4_profiles/EN.4.2.1.f.profiles.g10.200901.nc
21


IndexError: tuple index out of range

In [31]:
metadata.loc[profile_list[10]]['DEPH_CORRECTED'].values

array([  78.440575,   82.9078  ,   88.06981 ,   92.53683 ,   97.301544,
        102.36393 ,  106.9299  ,  111.69428 ,  121.12346 ,  130.5522  ,
        140.57597 ,  150.2023  ,  159.332   ,  169.65204 ,  179.6739  ,
        188.90152 ,  198.6248  ,  208.2484  ,  218.36758 ,  227.39507 ,
        237.5133  ,  247.13509 ,  256.75644 ,  265.98056 ,  285.7169  ,
        304.9555  ,  324.09317 ,  343.42734 ,  362.7597  ,  391.11057 ,
        420.05215 ,  457.8089  ,  505.66382 ,  553.9039  ,  602.5287  ,
        651.43915 ,  699.64526 ,  748.7307  ,  798.596   ,  847.1636  ,
        895.7199  ,  945.35223 ,  994.0833  , 1043.1982  , 1092.104   ,
       1141.591   , 1190.6714  , 1239.4441  , 1288.8962  , 1338.2382  ,
       1387.3711  , 1436.1967  , 1534.9971  , 1632.963   , 1731.2773  ,
       1829.2509  , 1926.9824  , 2024.3739  ], dtype=float32)

In [28]:
'''
Next step is to create the depthlevels files (completed; below) and the station files (completed; below) in the loop or in a new loop
'''

# depthlevels = np.arange(5, maxdepth[i], 5)
# depthlevels = np.append(depthlevels, maxdepth[i])
# f = open("DEPTHLEVEL_FILES/depthlevels.in."+str(stn[i])+"."+str(year[i])+"_"+str("%03d" % (doy[i])), "w")
# f.write(str(len(depthlevels))+"                  # Number of z levels"+"\n")
# for x in range(0, len(depthlevels)):
# f.write("%s\n" % depthlevels[x])

# f.close()

for profile in profile_list:
    print(profile)
    
    # Assign variables from profile data
    stn = profile
    date = pd.to_datetime(metadata.loc[profile]['JULD'][0])
    doy = date.dayofyear - 1
    year = date.year
    lon = metadata.loc[profile]['LONGITUDE'][0]
    lat = metadata.loc[profile]['LATITUDE'][0]
    depthlevels = metadata.loc[profile]['DEPH_CORRECTED'].values
    
    # Write depthlevels.in file
    depth_file = open("EN4_depth_station/depthlevels.in.{0}.{1}_{2:03}".format(stn,year,doy), "w")
    depth_file.write("{0}                  # Number of z levels\n".format(len(depthlevels)))
    for depth in depthlevels:
        depth_file.write("{0}\n".format(depth))
    depth_file.close()
    
    # Write stations.in file
    station_file = open("EN4_depth_station/stations.in.{0}.{1}_{2:03}".format(stn,year,doy), "w")
    station_file.write("#{0}\n".format(stn))
    station_file.write("{0:.2f}   {1:.2f}\n".format(lon,lat))
    station_file.close()


21
22
54
92
93
94
152
162
169
189
235
240
248
254
278
281
328
347
388
391
394
464
467
471
494
495
536
577
590
591
627
649
663
705
706
720
725
732
735
743
788
808
810
816
844
845
858
880
922
924
959
962
977
979
999
1000
1038
1040
1054
1064
1087
1138
1150
1155
1180
1182
1223
1254
1256
1281
1293
1301
1359
1366
1372
1402
1403
1428
1470
1481
1485
1510
1533
1544
1553
1559
1593
1595
1608
1612
1617
1619
1622
1675
1687
1691
1696
1701
1720
1722
1737
1764
1801
1808
1815
1817
1862
1901
1902
1936
1942
1957
1970
1978
1998
2049
2055
2057
2066
2095
2097
2129
2161
2179
2195
2207
2271
2273
2277
2304
2308
2333
2376
2378
2387
2424
2440
2449
2462
2476
2477
2508
2518
2523
2527
2528
2531
2577
2580
2584
2593
2600
2627
2630
2631
2668
2697
2708
2711
2747
2768
2776
2789
2795
2797
2800
2804
2807
2815
2819
2834
2840
2843
2846
2853
2863
2875
2877
2884
2886
2893
2902
2903
2908
2912
2931
2932
2949
2951
2954
2955
2965
2966
2967
2968
2973
2974
2975
2977
2984
2993
2996
2997
3008
3014
3020
3038
3039
3040
3045
3046
3054
3